<a href="https://colab.research.google.com/github/Maheshmrn/book-sphere/blob/main/FinalBookSphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.6 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics.pairwise import cosine_similarity
rating = pd.read_csv('/content/drive/MyDrive/Ratings.csv.zip')
books = pd.read_csv('/content/drive/MyDrive/Books.csv.zip')
rating_with_name = rating.merge(books,on='ISBN')
#popularity based code
num_rating = rating_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating.rename(columns={'Book-Rating' : 'num-rating'}, inplace = True)
avg_rating = rating_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating.rename(columns={'Book-Rating' : 'avg-rating'},inplace = True)
popular_books = num_rating.merge(avg_rating, on = 'Book-Title')
popular_books = popular_books[popular_books['num-rating']>=250].sort_values('avg-rating',ascending= False).head(50)
popular_books = popular_books.merge(books, on = 'Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num-rating','avg-rating']]
#collaborative based code
x = rating_with_name.groupby('User-ID').count()['Book-Rating'] >= 200
most_rating_users = x[x].index
filtered_rating = rating_with_name[rating_with_name['User-ID'].isin(most_rating_users)]
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index
final_rating = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
pt = final_rating.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pt.fillna(0, inplace=True)
similar_scores = cosine_similarity(pt)
# collabative-based function
def bookSphere(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_books = sorted(list(enumerate(similar_scores[index])), key=lambda x:x[1], reverse=True)[0:11]
    results = []
    for i in similar_books:
        books1 = books[books['Book-Title'] == pt.index[i[0]]]
        book_info = []
        book_info.extend(list(books1.drop_duplicates('Book-Title')['Book-Title'].values))
        book_info.extend(list(books1.drop_duplicates('Book-Title')['Book-Author'].values))
        book_info.extend(list(books1.drop_duplicates('Book-Title')['Image-URL-M'].values))
        results.append(book_info)
    return results
# Popularity-based function
def popular_books_list():
    results = []
    for title in popular_books['Book-Title']:
        books1 = books[books['Book-Title'] == title].drop_duplicates('Book-Title')
        book_info = []
        book_info.extend(list(books1['Book-Title'].values))
        book_info.extend(list(books1['Book-Author'].values))
        book_info.extend(list(books1['Image-URL-M'].values))
        results.append(book_info)
    return results
# Streamlit UI
st.set_page_config(page_title="📚 BookSphere", layout="wide")
if 'history' not in st.session_state:
    st.session_state.history = []
if 'rated_books' not in st.session_state:
    st.session_state.rated_books = []
if 'readlist' not in st.session_state:
    st.session_state.readlist = []
if 'selected_book' not in st.session_state:
    st.session_state.selected_book = ""
st.markdown("<h1 style='text-align: center; color: #4B0082;'>📚 BookSphere</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center; font-size:20px; color: gray;'>Smart AI-Powered Book Recommendation System</p>", unsafe_allow_html=True)
#Home and Search
col_top = st.columns([1, 5, 2])
with col_top[0]:
    if st.button("🏠 Home", use_container_width=True):
        st.session_state.selected_book = ""
        st.rerun()
with col_top[2]:
    book_list = pt.index.sort_values().tolist()
    selected_book = st.selectbox("🔍 Choose a Book:", options=[""] + book_list)
    if selected_book:
        st.session_state.selected_book = selected_book
        if selected_book not in st.session_state.history:
            st.session_state.history.append(selected_book)
    show_recommendation = st.button("🎯 Get Recommendations", use_container_width=True)
    if st.button("🗑️ Clear Search History", use_container_width=True):
        st.session_state.history = []
# Divider
st.markdown("---")
col_feat = st.columns(3)
with col_feat[0]:
    if st.button("📖 Books You've Rated", use_container_width=True):
        if st.session_state.rated_books:
            st.subheader("📖 Books You Have Rated")
            rated_df = pd.DataFrame(st.session_state.rated_books)
            cols = st.columns(3)
            delete_index = None
            for idx, row in rated_df.iterrows():
                with cols[idx % 3]:
                    st.image(row['Image'], width=150)
                    st.markdown(f"{row['Book Title']}")
                    full_stars = int(row['Rating'])
                    half_star = row['Rating'] - full_stars >= 0.5
                    stars_display = "⭐" * full_stars + ("✨" if half_star else "")
                    st.markdown(f"**Rating:** {stars_display} ({row['Rating']}/5)")
                    if st.button(f"❌ Remove Rated Book '{row['Book Title']}'", key=f"remove_rated_{idx}"):
                        delete_index = idx
            if delete_index is not None:
                st.session_state.rated_books.pop(delete_index)
                st.success("✅ Book removed from Rated List!")
        else:
            st.info("You haven't rated any books yet!")
with col_feat[1]:
    if st.button("📚 My Readlist", use_container_width=True):
        if st.session_state.readlist:
            st.subheader("📚 Your Reading List")
            read_df = pd.DataFrame(st.session_state.readlist)
            cols = st.columns(3)
            delete_index = None
            for idx, row in read_df.iterrows():
                with cols[idx % 3]:
                    st.image(row['Image'], width=150)
                    st.markdown(f"**{row['Book Title']}**")
                    st.markdown(f"*by {row['Author']}*")
                    if st.button(f"❌ Remove from Readlist '{row['Book Title']}'", key=f"remove_readlist_{idx}"):
                        delete_index = idx
            if delete_index is not None:
                st.session_state.readlist.pop(delete_index)
                st.success("✅ Book removed from Readlist!")
        else:
            st.info("Your Readlist is empty!")
with col_feat[2]:
    if st.button("📊 Show Ratings Distribution", use_container_width=True):
        if st.session_state.rated_books:
            st.subheader("📊 Your Ratings Distribution")
            rated_df = pd.DataFrame(st.session_state.rated_books)
            rating_counts = rated_df['Rating'].value_counts().sort_index()
            fig, ax = plt.subplots()
            ax.pie(rating_counts, labels=[f"{r} ⭐" for r in rating_counts.index], autopct='%1.1f%%', startangle=90)
            ax.axis('equal')
            st.pyplot(fig)
        else:
            st.info("You haven't rated any books yet!")
# Divider
st.markdown("---")
# Recommendations
if st.session_state.selected_book:
    try:
        recs = bookSphere(st.session_state.selected_book)
        st.subheader(f"📘 Books similar to '{st.session_state.selected_book}'")
        cols = st.columns(3)
        for idx, book in enumerate(recs):
            title, author, img = book
            avg = avg_rating[avg_rating['Book-Title'] == title]['avg-rating']
            avg_score = round(avg.values[0], 2) if not avg.empty else "N/A"
            with cols[idx % 3]:
                st.image(img, width=150)
                st.markdown(f"**{title}**")
                st.markdown(f"*by {author}*")
                st.markdown(f"⭐ **Average Rating:** {avg_score}")
                search_url = f"https://www.amazon.in/s?k={title.replace(' ', '+')}"
                st.markdown(f"[🔗 More Info]({search_url})", unsafe_allow_html=True)
                user_rating = st.slider(f"⭐ Rate '{title}'", 1.0, 5.0, step=0.5, key=f"rating_{idx}")
                if st.button(f"💾 Save Rating for '{title}'", key=f"save_rating_{idx}"):
                    rating_entry = {
                        'Book Title': title,
                        'Author': author,
                        'Image': img,
                        'Rating': user_rating
                    }
                    st.session_state.rated_books.append(rating_entry)
                    file_path = "user_feedback.csv"
                    feedback = pd.DataFrame([rating_entry])
                    if os.path.exists(file_path):
                        feedback.to_csv(file_path, mode='a', header=False, index=False)
                    else:
                        feedback.to_csv(file_path, index=False)
                    st.success(f"✅ Your rating for '{title}' has been saved!")
                if st.button(f"📚 Add to Readlist '{title}'", key=f"readlist_{idx}"):
                    read_entry = {
                        'Book Title': title,
                        'Author': author,
                        'Image': img
                    }
                    if read_entry not in st.session_state.readlist:
                        st.session_state.readlist.append(read_entry)
                        st.success(f"✅ '{title}' added to your Readlist!")
                    else:
                        st.info(f"ℹ️ '{title}' is already in your Readlist!")
    except Exception as e:
        st.error("Book not found or not enough data.")
        st.text(f"Error details: {e}")
st.markdown("---")
# Recently Searched
if st.session_state.history:
    st.markdown("### 🕘 Recently Searched:")
    for recent in reversed(st.session_state.history[-5:]):
        st.markdown(f"🔹 {recent}")
# Popular section
if not st.session_state.selected_book:
    st.subheader("🔥 Most Popular Books")
    pop_recs = popular_books.values.tolist()
    cols = st.columns(3)
    for idx, book in enumerate(pop_recs):
        title, author, img, _, avg_score = book
        with cols[idx % 3]:
            st.image(img, width=150)
            st.markdown(f"{title}")
            st.markdown(f"{author}")
            st.markdown(f"⭐ Average Rating:{round(avg_score, 2)}")
            search_url = f"https://www.amazon.in/s?k={title.replace(' ', '+')}"
            st.markdown(f"[🔗 More Info]({search_url})", unsafe_allow_html=True)
""")

In [12]:
!pkill streamlit
!killall ngrok

In [13]:
!ngrok authtoken 2vaf6AFGfMb9KmgkK1QjDn1mukl_5qP6QVzJtWiZeX9nar9G3
from pyngrok import ngrok
public_url = ngrok.connect("http://localhost:8501")
print("🚀 Your live app: ", public_url)
!streamlit run app.py &>/dev/null &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Your live app:  NgrokTunnel: "https://3a92-34-173-213-217.ngrok-free.app" -> "http://localhost:8501"
